In [ ]:
import os, shutil, random, pathlib
from collections import defaultdict

random.seed(1337)  # Fijamos la semilla para que las selecciones sean reproducibles.

src_dir = "PruebaDataset"            # Indicamos la carpeta de origen con subcarpetas por clase.
dst_dir = "PruebaDataset_balanced"   # Indicamos la carpeta de destino donde crearemos el dataset balanceado.
extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}  # Definimos las extensiones de imagen permitidas.

# 1) Recolectamos rutas por clase.
files_by_class = defaultdict(list)
for cls in sorted(os.listdir(src_dir)):
    cpath = os.path.join(src_dir, cls)
    if not os.path.isdir(cpath):
        continue  # Ignoramos entradas que no sean directorios de clase.
    for root, _, files in os.walk(cpath):
        for f in files:
            if pathlib.Path(f).suffix.lower() in extensions:
                files_by_class[cls].append(os.path.join(root, f))  # Añadimos la ruta de cada imagen válida.

# 2) Calculamos cuántas imágenes hay por clase y definimos N objetivo por clase.
counts = {cls: len(v) for cls, v in files_by_class.items()}
min_count = min(counts.values())  # Obtenemos el mínimo por si queremos usarlo como límite inferior.
N = 100  # Definimos el número de muestras por clase, ajustable según disponibilidad por clase.

print("Imágenes por clase:", counts)  # Mostramos el conteo de imágenes por clase para diagnóstico.
print(f"Usando N={N} por clase")  # Confirmamos el N elegido por clase.

# 3) Creamos la carpeta de destino limpia.
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)  # Eliminamos el destino si existía para evitar mezclas previas.
os.makedirs(dst_dir, exist_ok=True)  # Creamos la carpeta de destino desde cero.

# 4) Definimos helper para enlazar o copiar archivos según el sistema.
def link_or_copy(src, dst):
    try:
        os.symlink(os.path.abspath(src), dst)  # Creamos un enlace simbólico para ahorrar espacio cuando sea posible.
    except Exception:
        shutil.copy2(src, dst)  # Copiamos físicamente el archivo si no podemos crear el enlace.

# 5) Seleccionamos N imágenes aleatorias por clase y las escribimos en el destino.
for cls, paths in files_by_class.items():
    random.shuffle(paths)  # Barajamos las rutas para muestrear aleatoriamente.
    keep = paths[:N]  # Tomamos las primeras N rutas tras el barajado.
    out_cls = os.path.join(dst_dir, cls)
    os.makedirs(out_cls, exist_ok=True)  # Creamos la subcarpeta de la clase en el destino.
    for i, p in enumerate(keep):
        ext = pathlib.Path(p).suffix.lower()
        dst_path = os.path.join(out_cls, f"{cls}_{i:05d}{ext}")  # Generamos un nombre de archivo estable y ordenable.
        link_or_copy(p, dst_path)  # Enlazamos o copiamos la imagen al destino.

print("✓ Dataset balanceado creado en:", dst_dir)  # Indicamos finalización y la ruta del dataset balanceado.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import json

# -----------------------------------------------------------------------------
# 1) Configuración reproducible.
# -----------------------------------------------------------------------------
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Silenciamos los logs de TensorFlow para mantener la salida limpia.

# -----------------------------------------------------------------------------
# 2) Generadores de datos (consistentes con ResNet50V2).
# -----------------------------------------------------------------------------
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess

img_size = (256, 256)
batch_size = 32
target_shape = (*img_size, 3)

# Creamos el generador de datos con aumentación para el entrenamiento.
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    preprocessing_function=resnet_preprocess
)

# Creamos el generador de datos para validación solo con preprocesamiento.
valid_datagen = ImageDataGenerator(
    preprocessing_function=resnet_preprocess
)

# Cargamos las imágenes de entrenamiento desde la carpeta correspondiente.
data_gen_entrenamiento = train_datagen.flow_from_directory(
    "Dataset",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    class_mode="categorical",
    color_mode="rgb"
)

# Obtenemos los nombres de las clases y la cantidad total detectada.
class_names = list(data_gen_entrenamiento.class_indices.keys())
num_classes = len(class_names)
print("Clases detectadas en TRAIN:", num_classes)
print(class_names)

#with open("class_names.json", "w", encoding="utf-8") as f:
#    json.dump(class_names, f, ensure_ascii=False, indent=2)


# Creamos el generador de validación, forzando el mismo orden de clases.
data_gen_pruebas = valid_datagen.flow_from_directory(
    "Dataset",
    target_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=SEED,
    class_mode="categorical",
    color_mode="rgb",
    classes=class_names
)

# -----------------------------------------------------------------------------
# 3) Definición del modelo (ResNet50V2 base + capa de clasificación).
# -----------------------------------------------------------------------------
def crear_modelo_resnet(input_shape, n_classes):
    from tensorflow.keras.applications import ResNet50V2

    # Cargamos la base preentrenada sin la parte superior.
    base = ResNet50V2(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling="avg",
        name="resnet50v2"
    )
    base.trainable = False  # Mantenemos la base congelada inicialmente.

    # Definimos la arquitectura del modelo completo.
    inputs = keras.Input(shape=input_shape)
    x = base(inputs, training=False)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)

    # Compilamos el modelo con Adam y entropía cruzada categórica.
    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# -----------------------------------------------------------------------------
# 4) Creación del modelo.
# -----------------------------------------------------------------------------
model = crear_modelo_resnet(target_shape, num_classes)
model.summary()

# -----------------------------------------------------------------------------
# 5) Definición de callbacks.
# -----------------------------------------------------------------------------
callbacks_iniciales = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=8,
        restore_best_weights=True,
        min_delta=0.001
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-7
    )
]

# -----------------------------------------------------------------------------
# 6) Entrenamiento inicial con la base congelada.
# -----------------------------------------------------------------------------
print("🚀 Entrenamiento inicial...")
historial = model.fit(
    data_gen_entrenamiento,
    epochs=30,
    validation_data=data_gen_pruebas,
    callbacks=callbacks_iniciales,
    verbose=1
)

# -----------------------------------------------------------------------------
# 7) Fine-tuning: descongelamos las capas superiores de la base.
# -----------------------------------------------------------------------------
print("🔧 Fine-tuning...")

base_model = model.get_layer("resnet50v2")
N = 30  # Número de capas superiores a descongelar.
for layer in base_model.layers[-N:]:
    if not isinstance(layer, layers.BatchNormalization):
        layer.trainable = True

# Recompilamos con una tasa de aprendizaje más baja.
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks_ft = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=4,
        restore_best_weights=True
    )
]

historial_ft = model.fit(
    data_gen_entrenamiento,
    epochs=10,
    validation_data=data_gen_pruebas,
    callbacks=callbacks_ft,
    verbose=1
)

# -----------------------------------------------------------------------------
# 8) Evaluación final del modelo.
# -----------------------------------------------------------------------------
print("📊 Evaluación final en validación:")
val_loss, val_acc = model.evaluate(data_gen_pruebas, verbose=0)
print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

#model.save("modelo.h5")

# -----------------------------------------------------------------------------
# 9) Visualización de las curvas de entrenamiento.
# -----------------------------------------------------------------------------
acc = historial.history.get('accuracy', []) + historial_ft.history.get('accuracy', [])
val_acc = historial.history.get('val_accuracy', []) + historial_ft.history.get('val_accuracy', [])
loss = historial.history.get('loss', []) + historial_ft.history.get('loss', [])
val_loss_hist = historial.history.get('val_loss', []) + historial_ft.history.get('val_loss', [])

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(acc)
plt.plot(val_acc)
plt.title('Precisión del modelo.')
plt.ylabel('Precisión.')
plt.xlabel('Época.')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(loss)
plt.plot(val_loss_hist)
plt.title('Pérdida del modelo.')
plt.ylabel('Pérdida.')
plt.xlabel('Época.')
plt.legend(['Entrenamiento', 'Validación'], loc='upper left')

plt.tight_layout()
plt.show()

# -----------------------------------------------------------------------------
# 10) Cálculo de métricas por clase y matriz de confusión.
# -----------------------------------------------------------------------------
from sklearn.metrics import classification_report, confusion_matrix

# Obtenemos las predicciones sobre el conjunto de validación.
y_true = data_gen_pruebas.classes
y_prob = model.predict(data_gen_pruebas, verbose=1)
y_pred = np.argmax(y_prob, axis=1)

# Generamos el informe de clasificación por clase.
labels_all = np.arange(num_classes)

print("\n📄 Classification report por clase:")
print(classification_report(
    y_true,
    y_pred,
    labels=labels_all,
    target_names=class_names,
    digits=4,
    zero_division=0
))

# Calculamos la matriz de confusión.
cm = confusion_matrix(y_true, y_pred, labels=labels_all)
print("\n🔢 Matriz de confusión (filas=verdadero, columnas=predicho):")
print(cm)

# Calculamos la exactitud por clase.
cm_diag = np.diag(cm)
support = cm.sum(axis=1)
acc_per_class = cm_diag / np.maximum(support, 1)

print("\n✅ Exactitud por clase:")
for name, acc_c, sup in zip(class_names, acc_per_class, support):
    print(f" - {name:<20s} acc={acc_c:7.4f}  soporte={sup}")


Found 11850 images belonging to 120 classes.
Clases detectadas en TRAIN: 120
['Aguacate', 'Anacardos', 'Banana', 'Berberechos', 'Calabacín', 'Calabaza', 'Canela', 'Canónigos', 'Carne Cerdo', 'Carne Conejo', 'Carne Cordero', 'Carne Pavo', 'Carne Ternera', 'Carne picada', 'Chorizo', 'Cilantro', 'Clavo', 'Coliflor', 'Curry', 'Cuscus', 'Espaguetis', 'Espinacas', 'Fideos', 'Gambas', 'Garbanzos', 'Grosellas', 'Guisantes', 'Hierbabuena', 'Jamon iberico', 'Levadura', 'Lima', 'Limón', 'Mantequilla', 'Mayonesa', 'Mejillones', 'Melocotones', 'Melón', 'Membrillo', 'Merluza', 'Miel', 'Mora', 'Morcilla', 'Mostaza', 'Naranja', 'Nata líquida', 'Nueces', 'Nuez moscada', 'Níscalos', 'Orégano', 'Pechuga de pollo', 'Salmon', 'Tomillo', 'Zanahoria', 'aceitunas', 'acelgas', 'ajo', 'albahaca', 'alcachofa', 'almejas', 'almendras', 'apio', 'arroz', 'atun fresco', 'avellanas', 'azucar', 'bacalao', 'berenjena', 'boniato', 'boquerones', 'brocoli', 'calamares', 'cebollas', 'cerezas', 'champinones', 'chirimoya', 'c

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ resnet50v2 (Functional)              │ (None, 2048)                │      23,564,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 120)                 │         245,880 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,810,680 (90.83 MB)

 Trainable params: 245,880 (960.47 KB)

 Non-trainable params: 23,564,800 (89.89 MB)

🚀 Entrenamiento inicial...


C:\Users\Jose\anaconda3\envs\Proyecto\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
C:\Users\Jose\anaconda3\envs\Proyecto\Lib\site-packages\PIL\Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/30
 56/371 ━━━━━━━━━━━━━━━━━━━━ 9:01 2s/step - accuracy: 0.0402 - loss: 5.2050